### installing requirements

In [1]:
!pip install openai google-search-results taskflowai

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.9/252.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.7/294.7 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

## import apis, etc

In [ ]:
from taskflowai import OpenaiModels
from google.colab import userdata
import os

# ✅ Set API Keys (You should remove them after testing for security reasons)
os.environ["OPENAI_API_KEY"] = "your api key"
os.environ["EXA_API_KEY"] = "your api key"
os.environ["SERPER_API_KEY"] = "your api  key"


# ✅ Fetch API Keys from Colab Secrets or Environment Variables
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY") or os.getenv("OPENAI_API_KEY")
EXA_API_KEY = userdata.get("EXA_API_KEY") or os.getenv("EXA_API_KEY")
SERPER_API_KEY = userdata.get("SERPER_API_KEY") or os.getenv("SERPER_API_KEY")





# ✅ Validate API Keys
if not OPENAI_API_KEY:
    raise ValueError("❌ Missing OPENAI_API_KEY. Set it in Colab using `userdata` or `os.environ`.")

if not EXA_API_KEY:
    raise ValueError("❌ Missing EXA_API_KEY. Set it in Colab using `userdata` or `os.environ`.")

if not SERPER_API_KEY:
    raise ValueError("❌ Missing SERPER_API_KEY. Set it in Colab using `userdata` or `os.environ`.")



print("✅ API Keys Loaded Successfully!")


class LoadModel:
    @staticmethod
    def load_openai_model():
        """Load and return the OpenAI GPT-3.5-turbo model."""
        try:
            return OpenaiModels.gpt_3_5_turbo
        except Exception as e:
            raise RuntimeError(f"Error loading OpenAI model: {e}") from e


✅ API Keys Loaded Successfully!


### Wiki_Articles Tool

In [3]:
from taskflowai import WikipediaTools

class WikiArticles:
    @staticmethod
    def fetch_articles(query: str):
        try:
            articles = WikipediaTools.search_articles(query=query)
            return articles
        except Exception as e:
            raise RuntimeError(f"Error fetching Wikipedia articles: {e}") from e


### Wiki_Images Tool

In [4]:
from taskflowai import WikipediaTools

class WikiImages:
    @staticmethod
    def search_images(query: str):
        """Search for images on Wikipedia based on the given query."""
        try:
            return WikipediaTools.search_images(query=query)
        except Exception as e:
            raise RuntimeError(f"Error fetching Wikipedia images: {e}") from e


### EXa search APi tool

In [5]:
from taskflowai import WebTools
from typing import Optional

class ExaSearch:
    @staticmethod
    def search_web(query: str, num_results: Optional[int] = 5):
        """Perform a web search with a query and return the result."""
        try:
            if "EXA_API_KEY" not in os.environ:
                raise Exception("EXA_API_KEY environment variable is not set")

            search_result = WebTools.exa_search(query, int(num_results))
            return search_result

        except Exception as e:
            print(f"Search failed: {e}")
            return "No data available"


### EXA  Shopping Search tool



In [6]:
import os
from taskflowai import WebTools
from typing import Optional

class ExaShoppingSearch:
    @staticmethod
    def search_web(query: str, num_results: Optional[int] = 5):
        """Perform a web search for shopping results with a given query."""
        try:
            if "EXA_API_KEY" not in os.environ:
                raise Exception("EXA_API_KEY environment variable is not set")

            # Fix: Use 'queries' instead of 'query'
            search_result = WebTools.exa_search(
                queries=query,  # <- Corrected this line
                search_type="keyword",
                num_results=int(num_results)
            )
            return search_result

        except Exception as e:
            return f"Search failed: {e}"  # Return error message instead of printing


###  serper Shopping search tool

In [19]:
## Serper api was down during the creation of this project, so I gave agent the access to ExaShopping as second option:

import os
from dotenv import load_dotenv
from taskflowai import WebTools
from typing import Optional

class SerperSearch:
    @staticmethod
    def search_web(query: str, num_results: Optional[int] = 5):
        """Perform a web search using the Serper API.

        Args:
            query (str): The search query.
            num_results (Optional[int]): Number of results to fetch (default: 5).

        Returns:
            dict: The search results from the API.

        Raises:
            RuntimeError: If the search fails.
        """
        try:
            # Load environment variables from .env file
            load_dotenv()

            # Retrieve API key
            api_key = os.getenv("SERPER_API_KEY")
            if not api_key:
                raise ValueError("SERPER_API_KEY is not set in the environment or .env file.")

            # Perform web search
            return WebTools.serper_search(query=query, search_type="shopping", num_results=int(num_results))

        except Exception as e:
            raise RuntimeError(f"Failed to perform web Serper search: {e}") from e


### Web Research Agent

In [8]:
from taskflowai import Agent

class WebResearchAgent:
    @staticmethod
    def initialize_web_research_agent():
        """
        Initializes and returns the Crop or Plant Web Research Agent.
        """
        try:
            web_research_agent = Agent(
                role="Crop and Plant Research Agent",
                goal="Collect key details about a crop or plant, including classification, uses, growth conditions, and images.",
                attributes="Accurate, structured, and data-driven.",
                llm=LoadModel.load_openai_model(),
                tools=[
                    WikiArticles.fetch_articles,
                    WikiImages.search_images,
                    ExaSearch.search_web
                ],
            )

            return web_research_agent

        except Exception as e:
            raise Exception(f"Failed to initialize Web Research Agent: {str(e)}")

### Price Fetching Agent

In [21]:
from taskflowai import Agent

class PriceFetchingAgent:
    @staticmethod
    def initialize_price_fetching_agent(query: str):
        """
        Initializes and returns the Crop or Plant Price Research Agent.

        Args:
            query (str): The plant name or keyword for price research.

        Returns:
            Agent: A price research agent instance.
        """
        try:
            price_fetching_agent = Agent(
                role="Price Research Agent",
                goal="Find and compare the best prices for crops and plants across different markets.",
                attributes="Cost-conscious, data-driven, and focused on accurate price comparisons.",
                llm=LoadModel.load_openai_model(),
                tools=[SerperSearch.search_web, ExaShoppingSearch.search_web],
            )

            return price_fetching_agent

        except Exception as e:
            raise Exception(f"Error initializing Price Research Agent: {str(e)}")


### reseach overall web task

In [10]:
from taskflowai import Task

def research_overall_web(plant_name: str):
    """
    Conducts web research on a plant, gathering details on:
    - Scientific name, origin, and growing regions
    - Uses, benefits, and growth conditions
    - Common pests, diseases, and economic significance
    - Relevant images formatted as ![Description](https://full-image-url)

    Returns:
    - A structured research task with AI-powered web search.

    Raises:
    - Exception on failure.
    """

    try:
        agent = WebResearchAgent.initialize_web_research_agent()
        task = Task.create(
            agent=agent,
            context=f"Plant Name: {plant_name}",
            instruction=(
                f"Research {plant_name} and provide details on:\n"
                f"- Scientific classification, origin, and regions\n"
                f"- Uses, benefits, and growth conditions\n"
                f"- Pests, diseases, and economic significance\n"
                f"- Provide relevant images (![Description](https://full-image-url))\n"
            ),
        )
        return task

    except Exception as e:
        raise Exception(f"Error in research_overall_web for {plant_name}: {e}")


### Research health task

In [11]:
from taskflowai import Task

def research_health(plant_name: str):
    """Research health-related information about the plant."""
    try:
        agent = WebResearchAgent.initialize_web_research_agent()
        task = Task.create(
            agent=agent,
            context=f"Plant: {plant_name}",
            instruction=(
                f"Research health aspects of {plant_name}, including:\n"
                f"- Benefits & medicinal uses\n"
                f"- Risks & toxicity\n"
                f"- Nutritional value\n"
                f"- Traditional remedies\n"
                f"- Scientific studies\n"
                f"Provide structured, referenced insights."
            ),
        )
        return task
    except Exception as e:
        raise Exception(f"Error in research_health: {e}")


### Research Season task

In [12]:
from taskflowai import Task

def research_season(plant_name: str):
    """Research seasonal growth and farming details of the plant."""
    try:
        agent = WebResearchAgent.initialize_web_research_agent()
        task = Task.create(
            agent=agent,
            context=f"Plant: {plant_name}",
            instruction=(
                f"Research {plant_name}'s optimal growth conditions:\n"
                f"- Planting & harvesting seasons\n"
                f"- Climate, temperature, humidity\n"
                f"- Soil, nutrients, fertilizers\n"
                f"- Best farming practices\n"
                f"- Off-season storage & uses\n"
                f"Provide expert-backed agricultural insights."
            ),
        )
        return task
    except Exception as e:
        raise Exception(f"Error in research_season: {e}")


### price research task

In [13]:
from taskflowai import Task

def research_price(plant_name: str):
    """Research market price details of the plant, including the lowest available price."""
    try:
        agent = PriceFetchingAgent.initialize_price_fetching_agent(query=plant_name)
        task = Task.create(
            agent=agent,
            context=f"Plant: {plant_name}",
            instruction=(
                f"Fetch {plant_name} market prices:\n"
                f"- Online vs. offline price comparisons\n"
                f"- Cost per kg vs. per pound (by quality)\n"
                f"- Price differences by region (prioritize user location)\n"
                f"- Factors influencing price (season, demand, supply, quality)\n"
                f"- Identify the **lowest available price** and where it is found\n"
                f"Provide accurate, up-to-date market data."
            ),
        )
        return task
    except Exception as e:
        raise Exception(f"Error in research_price: {e}")


## Execute Research and Report - flow 1

In [14]:
def execute_research_and_report(plant_names):
    """Executes research tasks and generates a structured report for each plant.

    Args:
        plant_names (list): A list of plant names to research.

    Returns:
        dict: Research results categorized by plant name.
    """
    research_results = {}
    try:
        if not plant_names:
            return research_results

        for plant_name in plant_names:
            research_results[plant_name] = {
                "general": research_overall_web(plant_name),
                "health": research_health(plant_name),
                "season": research_season(plant_name),
                "price": research_price(plant_name),
            }

        return research_results

    except Exception as e:
        raise Exception(f"Error in execute_research_and_report: {str(e)}")


## Generate Summarized Report

In [15]:
def generate_summarized_report(research_results):
    """Generates a structured and easy-to-read summarized report from research results.

    Args:
        research_results (dict): A dictionary containing research data for each plant.

    Returns:
        str: A formatted summary report of the research.
    """
    try:
        if not research_results:
            return "No research results to summarize."

        summarized_report = "🌿 **Plant Research Summary** 🌿\n\n"

        for plant, results in research_results.items():
            summarized_report += f"🌱 **{plant.capitalize()}**\n"
            summarized_report += f"📌 **General Information:** {results.get('general', 'No data available')}\n"
            summarized_report += f"🩺 **Health Benefits & Risks:** {results.get('health', 'No data available')}\n"
            summarized_report += f"🌤 **Growing Season & Conditions:** {results.get('season', 'No data available')}\n"
            summarized_report += f"💰 **Market Prices & Trends:** {results.get('price', 'No data available')}\n"
            summarized_report += "-" * 50 + "\n\n"

        return summarized_report

    except Exception as e:
        raise Exception(f"Error in generate_summarized_report: {str(e)}")


### Endpoint

In [22]:
if __name__ == "__main__":
    plant_name = input("Enter the plant name(s) (comma-separated if multiple): ").strip()

    # Convert input into a list of plant names
    plant_names = [name.strip() for name in plant_name.split(",")]

    try:
        # Execute research and collect results
        research_results = execute_research_and_report(plant_names)

        # Print research results one by one as they are processed
        for plant, results in research_results.items():
            print(f"\n--- Research Results for {plant} ---")
            print("General Information:\n", results.get("general", "No data available"))
            print("\nHealth Research:\n", results.get("health", "No data available"))
            print("\nSeasonal Research:\n", results.get("season", "No data available"))
            print("\nPrice Research:\n", results.get("price", "No data available"))
            print("\n" + "-" * 40)  # Separator for clarity

        # Generate and print summarized report
        summarized_report = generate_summarized_report(research_results)
        print("\nFinal Summarized Report:\n")
        print(summarized_report)

    except Exception as e:
        print("\nAn error occurred:", str(e))


Enter the plant name(s) (comma-separated if multiple): chamomile
✔ Request completed
Tool Use: search_web
Parameters:
  query: chamomile scientific classification, origin, regions

Searching Exa for: 'chamomile scientific classification, origin, regions'
Result: {'queries': ['chamomile scientific classification, origin, regions'], 'results': [{'query': 'chamomile scientific classification, origin, regions', 'data': [{'title': 'Jammu and Kashmir Medicinal Plants Introduction Centre', 'url': 'https://jkmpic.blogspot.com/2020/11', 'author': '', 'highlights': "yellow. 'Jade Butterfly' - An unusual dwarf, slow-growing form, this plant has bright green leaves a...

Tool Use: search_web
Parameters:
  query: chamomile uses, benefits, growth conditions

Searching Exa for: 'chamomile uses, benefits, growth conditions'
Result: {'queries': ['chamomile uses, benefits, growth conditions'], 'results': [{'query': 'chamomile uses, benefits, growth conditions', 'data': [{'title': 'Chamomile: Usefulness 